In [1]:
!pip install -e . -q --upgrade
!pip install git+https://github.com/khursani8/minimalaya sentencepiece

  Cloning https://github.com/khursani8/minimalaya to /tmp/pip-req-build-iy3128vq
  Running command git clone --filter=blob:none --quiet https://github.com/khursani8/minimalaya /tmp/pip-req-build-iy3128vq
  Resolved https://github.com/khursani8/minimalaya to commit 246372510a307cf8ec7a58e3e60dd11eaa1c0c93
  Preparing metadata (setup.py) ... done


In [2]:
import soundfile as sf
from teeteeass.infer import TTS
from IPython.display import Audio
import numpy as np

In [3]:
def intersperse(lst, item):
    result = [item] * (len(lst) * 2 + 1)
    result[1::2] = lst
    return result

In [14]:
text = 'Kau orang kena try untuk buka puasa hari ni Selamat berbuka.'
language = "EN"

In [15]:
# text = text.split(" ")[0]

In [16]:
speakers = TTS.list_speakers()
print(speakers)

['Abercrombie', 'Abukuma', 'Acasta', 'Achilles', 'Admiral Graf Spee', 'Admiral Hipper', 'Agano', 'Ajax', 'Akagi', 'Akagi-chan', 'Akagi_Anchorage', 'Akashi', 'Akatsuki', 'Akizuki Ritsuko', 'Alabama', 'Albacore', 'Albion', 'Alfredo Oriani', 'Algérie', 'Allen M', 'Amagi', 'Amagi-chan', 'Amami Haruka', 'Amazon', 'An shan', 'Anchorage', 'Andrea Doria', 'Aoba', 'Aquila', 'Arashio', 'Archerfish', 'Ardent', 'Arethusa', 'Argus', 'Ariake', 'Arizona', 'Arizona META', 'Ark Royal', 'Ark Royal META', 'Arkhangelsk', 'Asashio', 'Ashigara', 'Astoria', 'Asukagawa Chise', 'Atago', 'Atlanta', 'Attilio Regolo', 'August von Parseval', 'Aulick', 'Aurora', 'Avrora', 'Ayanami', 'Aylwin', 'Azuma', 'Bache', 'Bailey', 'Baltimore', 'Bataan', 'Beagle', 'Belfast', 'Bellona', 'Béarn', 'KA', 'KAv2', 'Ägir']


In [17]:
speaker = speakers[-2]
# speaker= None
print(speaker)

KAv2


In [18]:
tts = TTS(speaker=speaker)

In [19]:
norm_text, phone, tone, word2ph  = tts.clean_text(
    text,
    language,
)

[['Kau'], ['or', 'ang'], ['ken', 'a'], ['try'], ['untuk'], ['b', 'uka'], ['pu', 'asa'], ['hari'], ['ni'], ['Sela', 'mat'], ['be', 'r', 'buka'], ['.']]
['_', 'k', 'ah', 'uw', 'ow', 'r', 'ae', 'ng', 'k', 'eh', 'n', 'aa', 't', 'r', 'iy', 'uh', 'n', 't', 'uw', 'k', 'b', 'uw', 'k', 'ah', 'p', 'w', 'aa', 's', 'aa', 'hh', 'aa', 'r', 'ih', 'n', 'ih', 's', 'eh', 'l', 'aa', 'm', 'ae', 't', 'b', 'er', 'b', 'uw', 'k', 'ah', '.', '_']


In [20]:
from teeteeass.nlp import SYMBOLS
print(SYMBOLS)

['_', 'AA', 'E', 'EE', 'En', 'N', 'OO', 'V', 'a', 'a:', 'aa', 'ae', 'ah', 'ai', 'an', 'ang', 'ao', 'aw', 'ay', 'b', 'by', 'c', 'ch', 'd', 'dh', 'dy', 'e', 'e:', 'eh', 'ei', 'en', 'eng', 'er', 'ey', 'f', 'g', 'gy', 'h', 'hh', 'hy', 'i', 'i0', 'i:', 'ia', 'ian', 'iang', 'iao', 'ie', 'ih', 'in', 'ing', 'iong', 'ir', 'iu', 'iy', 'j', 'jh', 'k', 'ky', 'l', 'm', 'my', 'n', 'ng', 'ny', 'o', 'o:', 'ong', 'ou', 'ow', 'oy', 'p', 'py', 'q', 'r', 'ry', 's', 'sh', 't', 'th', 'ts', 'ty', 'u', 'u:', 'ua', 'uai', 'uan', 'uang', 'uh', 'ui', 'un', 'uo', 'uw', 'v', 'van', 've', 'vn', 'w', 'x', 'y', 'z', 'zh', 'zy', '!', '?', '…', ',', '.', "'", '-', 'SP', 'UNK']


In [21]:
for k in ["?"]:
    norm_text, phone, tone, word2ph  = tts.clean_text(
        text,
        language,
    )
    phone[1] = k
    print(phone)
    phone, tone, lang_ids = tts.clean_text_to_sequence(
        phone,
        tone,
        language
    )
    phone = intersperse(phone, 0)
    tone = intersperse(tone, 0)
    lang_ids = intersperse(lang_ids, 0)
    for i in range(len(word2ph)):
        word2ph[i] = word2ph[i] * 2
    word2ph[0] += 1
    phones = np.longlong(phone)
    tones = np.longlong(tone)
    lang_ids = np.longlong(lang_ids)
    bert_ori = tts.get_bert_feature(norm_text,word2ph,language,'cpu')
    ja_bert = np.zeros((1024, len(phone)))
    en_bert = np.zeros((1024, len(phone)))
    bert    = np.zeros((1024, len(phone)))
    
    if language == "ZH":
        bert = bert_ori
    elif language == "JP":
        ja_bert = bert_ori
    elif language == "EN":
        en_bert = bert_ori
    else:
        raise ValueError("language should be ZH, JP or EN")
    audio = tts.session(phones, tones, lang_ids, bert.T, ja_bert.T, en_bert.T).squeeze()
    sf.write('test.wav', audio, 44100)
    print(k)
    display(Audio('test.wav'))

[['Kau'], ['or', 'ang'], ['ken', 'a'], ['try'], ['untuk'], ['b', 'uka'], ['pu', 'asa'], ['hari'], ['ni'], ['Sela', 'mat'], ['be', 'r', 'buka'], ['.']]
['_', 'k', 'ah', 'uw', 'ow', 'r', 'ae', 'ng', 'k', 'eh', 'n', 'aa', 't', 'r', 'iy', 'uh', 'n', 't', 'uw', 'k', 'b', 'uw', 'k', 'ah', 'p', 'w', 'aa', 's', 'aa', 'hh', 'aa', 'r', 'ih', 'n', 'ih', 's', 'eh', 'l', 'aa', 'm', 'ae', 't', 'b', 'er', 'b', 'uw', 'k', 'ah', '.', '_']
['_', '?', 'ah', 'uw', 'ow', 'r', 'ae', 'ng', 'k', 'eh', 'n', 'aa', 't', 'r', 'iy', 'uh', 'n', 't', 'uw', 'k', 'b', 'uw', 'k', 'ah', 'p', 'w', 'aa', 's', 'aa', 'hh', 'aa', 'r', 'ih', 'n', 'ih', 's', 'eh', 'l', 'aa', 'm', 'ae', 't', 'b', 'er', 'b', 'uw', 'k', 'ah', '.', '_']
?
